# Text Analytics for health

https://learn.microsoft.com/en-us/azure/ai-services/language-service/text-analytics-for-health/overview?tabs=ner?wt.mc_id=MVP_322781

## Install Library

In [ ]:
%pip install azure-ai-textanalytics

## Load Azure Configurations

In [1]:
import os

# Load Azure configurations from environment variables
# Ensure that AZURE_AI_LANGUAGE_KEY and AZURE_AI_LANGUAGE_ENDPOINT are set in your environment
language_key = os.environ.get('AZURE_AI_LANGUAGE_KEY')
language_endpoint = os.environ.get('AZURE_AI_LANGUAGE_ENDPOINT')

## Create a Text Analysis client

In [2]:
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

# Authenticate the client using Azure Key and Endpoint
def authenticate_client():
    """
    Authenticates the Azure Text Analytics client using the provided key and endpoint.

    Returns:
        TextAnalyticsClient: An authenticated client for Azure Text Analytics.
    """
    ta_credential = AzureKeyCredential(language_key)
    text_analytics_client = TextAnalyticsClient(
        endpoint=language_endpoint,
        credential=ta_credential
    )
    return text_analytics_client

# Initialize the client
client = authenticate_client()

## Begin Analyze Healthcare Entities function

In [10]:
def analyze_healthcare_entities(client, documents):
    """
    Analyzes healthcare entities in the provided documents using Azure Text Analytics.

    Args:
        client (TextAnalyticsClient): The Azure Text Analytics client.
        documents (list of str): A list of documents to analyze.

    Returns:
        None: Prints the extracted healthcare entities and their details.
    """
    # Start the healthcare entity analysis process
    poller = client.begin_analyze_healthcare_entities(documents)
    result = poller.result()
    
    # Filter out documents that encountered errors
    docs = [doc for doc in result if not doc.is_error]

    # Iterate through each document in the results
    for i, doc in enumerate(docs):
        # Extract the document number from the original text (if available)
        document_text = documents[i]
        doc_number = document_text.split("|")[0].strip() if "RECORD #" in document_text else f"Document {i + 1}"
        print(f"\nProcessing {doc_number}:\n")
        
        # Iterate through each entity in the document
        for entity in doc.entities:
            print(f"Entity: {entity.text}")
            print(f"...Normalized Text: {entity.normalized_text}")
            print(f"...Category: {entity.category}")
            print(f"...Subcategory: {entity.subcategory}")
            print(f"...Offset: {entity.offset}")
            print(f"...Confidence score: {entity.confidence_score}")
            
            # Print data sources if available
            if entity.data_sources is not None:
                print("...Data Sources:")
                for data_source in entity.data_sources:
                    print(f"......Entity ID: {data_source.entity_id}")
                    print(f"......Name: {data_source.name}")
            
            # Print assertion details if available
            if entity.assertion is not None:
                print("...Assertion:")
                print(f"......Conditionality: {entity.assertion.conditionality}")
                print(f"......Certainty: {entity.assertion.certainty}")
                print(f"......Association: {entity.assertion.association}")
        
        # Print relationships between entities
        for relation in doc.entity_relations:
            print(f"Relation of type: {relation.relation_type} has the following roles")
            for role in relation.roles:
                print(f"...Role '{role.name}' with entity '{role.entity.text}'")

In [18]:
documents = [
    """
    RECORD #50348590385904 | MH | 85986313 | | 054351 | 2/14/2001 12:00:00 AM |
    MEDICATIONS | Signed | DIS | Prescription Date: 4/10/2025 |
    Report Status: Signed Prescription Details:
    Patient needs to take:
    - 100 mg of ibuprofen
    - 3 mg of potassium
    - 10 mg of Zocor
    """,
    
    """
    RECORD #333582770390100 | MH | 85986313 | | 054351 | 2/14/2001 12:00:00 AM |
    CORONARY ARTERY DISEASE | Signed | DIS | Admission Date: 5/22/2001 |
    Report Status: Signed Discharge Date: 4/24/2001 ADMISSION DIAGNOSIS:
    CORONARY ARTERY DISEASE. HISTORY OF PRESENT ILLNESS:
    The patient is a 54-year-old gentleman with a history of progressive angina over the past several months.
    The patient had a cardiac catheterization in July of this year revealing total occlusion of the RCA and
    50% left main disease, with a strong family history of coronary artery disease with a brother dying at
    the age of 52 from a myocardial infarction and another brother who is status post coronary artery bypass grafting.
    The patient had a stress echocardiogram done on July, 2001, which showed no wall motion abnormalities,
    but this was a difficult study due to body habitus. The patient went for six minutes with minimal ST depressions
    in the anterior lateral leads, thought due to fatigue and wrist pain, his anginal equivalent. Due to the patient's
    increased symptoms and family history and history left main disease with total occlusion of his RCA was referred
    for revascularization with open heart surgery.
    """,

    """
    RECORD #123456789012345 | MH | 98765432 | | 123456 | 3/10/2025 12:00:00 PM |
    LIVER CIRRHOSIS | Signed | DIS | Admission Date: 3/5/2025 |
    Report Status: Signed Discharge Date: 3/15/2025 ADMISSION DIAGNOSIS:
    LIVER CIRRHOSIS. HISTORY OF PRESENT ILLNESS:
    The patient is a 45-year-old female with a history of chronic alcohol use and fatigue over the past year.
    She presented with abdominal pain, jaundice, and significant weight loss. Laboratory tests revealed elevated
    liver enzymes and bilirubin levels. An abdominal ultrasound showed evidence of liver cirrhosis with splenomegaly.
    The patient has no known family history of liver disease but reports a history of heavy alcohol consumption
    for the past 15 years. A liver biopsy confirmed the diagnosis of cirrhosis. During her hospital stay, she was
    managed with diuretics for ascites and lactulose for hepatic encephalopathy. She was counseled on alcohol cessation
    and referred to a liver transplant center for further evaluation.
    """
]

In [19]:
analyze_healthcare_entities(client, documents)


Processing RECORD #50348590385904:

Entity: 2/14/2001
...Normalized Text: None
...Category: Date
...Subcategory: None
...Offset: 57
...Confidence score: 1.0
Entity: 12:00:00 AM
...Normalized Text: None
...Category: Time
...Subcategory: None
...Offset: 67
...Confidence score: 0.99
Entity: 4/10/2025
...Normalized Text: None
...Category: Date
...Subcategory: None
...Offset: 133
...Confidence score: 1.0
Entity: 100 mg
...Normalized Text: None
...Category: Dosage
...Subcategory: None
...Offset: 226
...Confidence score: 0.99
Entity: ibuprofen
...Normalized Text: ibuprofen
...Category: MedicationName
...Subcategory: None
...Offset: 236
...Confidence score: 1.0
...Data Sources:
......Entity ID: C0020740
......Name: UMLS
......Entity ID: 0000019879
......Name: AOD
......Entity ID: M01AE01
......Name: ATC
......Entity ID: 0046165
......Name: CCPSS
......Entity ID: 0000006519
......Name: CHV
......Entity ID: 2270-2077
......Name: CSP
......Entity ID: DB01050
......Name: DRUGBANK
......Entity ID: